In [8]:
# Import libraries
import requests
import pandas as pd
import json

from bs4 import BeautifulSoup

BASE_URL = 'https://www.topuniversities.com'

In [3]:
# Make the request
r = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508079297350')
json_result = r.json()

In [4]:
def getUniInfo(URL):
    #International faculty member

    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    #Extract the div with the intels about the faculty
    facultyDiv  = soup.find("div", class_="faculty-main wrapper col-md-4")
    
    #In the case the data is not on the page
    if facultyDiv is None:
        facultyInt = -1
        facultyTot = -1
    else:
        facultyInt = extract_number(facultyDiv.find("div", class_="inter faculty"))
        facultyTot = extract_number(facultyDiv.find("div", class_="total faculty"))
    
    #Extract the div with the intels about the student

    studentInt = soup.find("div", class_="total inter")
    studentTot = soup.find("div", class_="total student")
    #In the case the data is not on the page
    
    if studentInt is None:
        studentInt = -1
    else:
        studentInt = extract_number(studentInt)
        
    if studentTot is None:
        studentTot = -1
    else:
        studentTot = extract_number(studentTot)
        
    return facultyInt,facultyTot,studentInt,studentTot

def extract_number(t):
    #Answer on how to extract the int out of string found here:
    #https://stackoverflow.com/questions/26825729/extract-number-from-string-python
    
    #In case the faculty doesn't have international faculty member
    if t is None:
        return -1
    
    return int(''.join(filter(str.isdigit, t.find("div", class_="number").text)))

In [5]:
#Building the dataFrame
df = pd.DataFrame(columns = ['Rank','Name','Country','Region','FacultyMemberInt','FacultyMemberTot','StudentInt','StudentTot'])

nb = 0
for i in json_result.keys():
    for x in json_result[i]:
        
        if nb == 200:
            break
        
        #print(x['title'])
        nb = nb + 1
        facultyInt,facultyTot,studentInt,studentTot = getUniInfo(BASE_URL + x['url'])
        current_uni = [x['rank_display'],x['title'],x['country'],x['region'],facultyInt,facultyTot,studentInt,studentTot]
        df.loc[len(df)] = current_uni


NameError: name 'display' is not defined

In [11]:
display(df)

NameError: name 'display' is not defined

In [38]:
def getBestRatio(data,col1,col2):
    """Compute and return the best ratio col1 / col2.FilteringBy indicate the column header we want to filter by"""
            
    bestRatio = 0
    bestName = ''
    
    for index, row in data.iterrows():

        if row[col1] > 0 and row[col2] > 0:
            currentRatio = row[col1] / row[col2]
            if bestRatio < currentRatio:
                bestRatio = currentRatio
                bestName = row['Name']
                
                
    return bestName,bestRatio  

In [39]:
#Get the best ratio of international faculty member.
uniWithBestFacRatio, maxIntFacRatio = getBestRatio(df,'FacultyMemberInt','FacultyMemberTot')
print("Name = {n}, Ratio = {r}".format(n=uniWithBestFacRatio,r=maxIntFacRatio))   

#Get the best ratio of international student.
uniWithBestStudRatio, maxIntStudRatio = getBestRatio(df,'StudentInt','StudentTot')
print("Name = {n}, Ratio = {r}".format(n=uniWithBestStudRatio,r=maxIntStudRatio)) 

Name = Ecole Polytechnique Fédérale de Lausanne (EPFL), Ratio = 0.7669616519174042
Name = London School of Economics and Political Science (LSE), Ratio = 0.6913934426229508


In [52]:
#Get the list of the countries in the dataFrame
countryStats = pd.DataFrame(columns = ['Name','FacultyMemberInt','FacultyMemberTot','StudentInt','StudentTot'])
countries = df.Country.unique()

for country in countries:
    
    #Delete the row with -1
    currentCountry = df[(df.Country == country)&(df.StudentInt >=0)&(df.StudentTot >=0)&(df.FacultyMemberInt >=0)&(df.FacultyMemberTot >=0)]
    
    #Sum the columns
    IntStud = currentCountry.StudentInt.sum()
    TotStud = currentCountry.StudentTot.sum()
    IntFac = currentCountry.FacultyMemberInt.sum()
    TotFac = currentCountry.FacultyMemberTot.sum()
    
    currentCountry = [country,IntFac,TotFac,IntStud,TotStud]
    countryStats.loc[len(countryStats)] = (currentCountry)
    
CountryWithBestFacRatio, maxIntFacCountryRatio = getBestRatio(countryStats,'FacultyMemberInt','FacultyMemberTot')
print("Name = {n}, Ratio = {r}".format(n=CountryWithBestFacRatio,r=maxIntFacCountryRatio)) 

CountryWithBestStudRatio, maxIntStudCountryRatio = getBestRatio(countryStats,'FacultyMemberInt','FacultyMemberTot')
print("Name = {n}, Ratio = {r}".format(n=CountryWithBestFacRatio,r=maxIntFacCountryRatio)) 

Name = Singapore, Ratio = 0.6436891147818721
